# **Question 3** Count records *

How many taxi trips were there on *January 15*?


```sql
SELECT
	COUNT(*)
FROM 
	yellow_taxi_trips
WHERE 
	DATE(tpep_pickup_datetime) = '2021-01-15'
```

**Answer**: 53024

# Question 4 Largest tip for each day *

On which day it was the largest tip in January? (*note: it's not a typo, it's "tip", not "trip"*)

```sql
WITH max_tips AS (
    SELECT 
		DATE(tpep_pickup_datetime) as d, max(tip_amount) as max_tip
	FROM 
		yellow_taxi_trips
	GROUP BY 
		d
   )

SELECT 
	DATE(tpep_pickup_datetime)
FROM 
	yellow_taxi_trips
WHERE 
	tip_amount = (SELECT 
					MAX(max_tip)
				 FROM 
					max_tips) 
```

**Answer**: 2021-01-20

# Question 5: Most popular destination *

*What was the most popular destination for passengers picked up in central park on January 14? Enter the zone name (not id). If the zone name is unknown (missing), write "Unknown"*


```sql
WITH max_value as (SELECT
				   		"DOLocationID" as dest_code, COUNT("DOLocationID") as counted_value
				   FROM 
				   		yellow_taxi_trips
				   WHERE 
				   		"PULocationID" = 43 and DATE(tpep_pickup_datetime) = '2021-01-14'
				   GROUP BY 
				   		"DOLocationID"
				   ORDER BY 
				   		counted_value DESC
				   LIMIT 
				   		1
				  )
                  
SELECT 
	dest_code
FROM 
	max_value
```



Here i use pandas to find area, here is a code:
    
```python
df = pd.read_csv('./taxi+_zone_lookup.csv')

df[df['Zone'] == 'Central Park']
```
i find LocationId code and use it in my query

|LocationID|Borough|Zone|service_zone
|-|-|-|-|
|43|Manhattan|Central Park|Yellow Zone

Than when i found zone and search what is it name by LocarionID

```python
df[df['LocationID'] == 237]
```

|LocationID|Borough|Zone|service_zone
|-|-|-|-|
|237|Manhattan|Upper East Side South|Yellow Zone

**Answer is** Upper East Side South*

# Another version

```SQL
WITH max_value as (SELECT
				   		"DOLocationID" as dest_code, COUNT("DOLocationID") as counted_value
				   FROM 
				   		yellow_taxi_trips
				   WHERE 
				   		"PULocationID" = 43 and DATE(tpep_pickup_datetime) = '2021-01-14'
				   GROUP BY 
				   		"DOLocationID"
				   ORDER BY 
				   		counted_value DESC
				   LIMIT 
				   		1
				  )
                  
SELECT 
	"Zone"
FROM 
	zone_lookup
WHERE "LocationID" = (SELECT dest_code FROM max_value)
```

# Question 6: Most expensive route *

What's the pickup-dropoff pair with the largest average price for a ride (calculated based on total_amount)? Enter two zone names separated by a slashFor example:"Jamaica Bay / Clinton East"If any of the zone names are unknown (missing), write "Unknown". For example, "Unknown / Clinton East".

```sql
SELECT 
	dest_code
FROM 
	max_value

SELECT 
	"PULocationID" as pickup, 
	"DOLocationID" as distantion,
	AVG(total_amount) avg_amount
FROM
	yellow_taxi_trips
GROUP BY
	"PULocationID", "DOLocationID"
ORDER BY
	avg_amount DESC
LIMIT 1
```

**Answer:**  LocationID 4 and  LocationID 265 ( Alphabet City and Unknown)

# Another version

```SQL
WITH most_expensive as (SELECT 
							"PULocationID" as pickup, 
							"DOLocationID" as distantion,
							AVG(total_amount) avg_amount
						FROM
							yellow_taxi_trips
						GROUP BY
							"PULocationID", "DOLocationID"
						ORDER BY
							avg_amount DESC
						LIMIT 1)

                  
SELECT 
	f."Zone" PickupZone,
	s."Zone" DeliveryZone
FROM 
	zone_lookup f, 
	zone_lookup s
WHERE 
    f."LocationID" in (SELECT pickup from most_expensive) AND
    s."LocationID" in (SELECT distantion from most_expensive)
```